# TAL aplicado al análisis del discurso de los medios de prensa 📰🤓🔥


El proyecto consiste en entrenar y evaluar varios modelos de clasificación supervisada capaz de clasificar una noticia según la taxonomía siguiente: 


- mundo = 0 
- economía = 1 
- política y conflictos = 2 
- ciencia y tecnología = 3
- catástrofes y accidentes = 4 
- cultura y arte = 5 
- deporte = 6 
- ecología y planeta = 7
- crimen, delitos y justicia = 8 
- salud = 9

- Hito Unidad 1 (29 de septiembre): Datasets de entrenamiento y test + primer modelo baseline

- Hito Unidad 2 (27 de octubre): Implementación y experimentos de varios modelos de clasificación

- Hito Proyecto (15 de diciembre): Evaluación y comparación de los modelos de los distintos equipos + integración de los mejores modelos en la arquitectura Sophia2.


### índex

1. [Importación del dataset](a)
2. [Extracción de tópicos vía RE en URLs](b)
3. [Extracción de términos clave de columnas title+text](b)
4. [Clasificación del dataset de entrenamiento](c)
5. [Balance de clases dataset de entrenamiento](d)
6. [Importación de los dataset restantes](e)
7. [Dividir dataset en entrenamiento y test](f)
8. [Dividir dataset entrenamiento en entrenamiento y validación](g)

## Generando dataset de entrenamiento

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Data manipulation
import re
import multiprocessing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# NLP
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec, CoherenceModel
#import pyLDAvis
#import pyLDAvis.gensim_models  # don't skip this
import nltk 
#nltk.download('stopwords') 
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("es_core_news_sm")
print("versión de Spacy: ", spacy.__version__)

versión de Spacy:  3.1.3


### 1. Importación de dataset <a name="a"></a>

In [2]:
dataset = pd.read_csv("data/chile_2020-09.csv")
dataset['content'] = dataset['title'] + " " + dataset['text'] 

print("Largo de dataset: ", len(dataset))
for index, row in dataset.iterrows():
    if pd.isna(row['text']):
        dataset.drop(index, inplace=True)
        
dataset.drop(columns=['country','year','date','id', 'id_journalist','title','text'], inplace=True)

dataset['value'] = ""

Largo de dataset:  13965


### 2.  Extracción de tópicos vía RE en URL <a name="b"></a>

In [3]:
for index, row in dataset.iterrows():
    url=row['url']
    obj = re.findall('(\w+)://([\w\-\.]+)/([\w\-]+).([\w\-]+)', url) 
    topic=obj[0][2]
    dataset.loc[index,'topic'] = topic
    
    if topic == 'mundo': 
        row['value'] = 0
    if topic == 'economia': 
        row['value'] = 1
    if topic == 'negocios': 
        row['value'] = 1      
    if topic == 'politica': 
        row['value'] = 2
    if topic == 'ciencia': 
        row['value'] = 3
    if topic == 'tecnologia': 
        row['value'] = 3    
    if topic == 'catastrofes': 
        row['value'] = 4      
    if topic == 'cultura': 
        row['value'] = 5
    if topic == 'artes': 
        row['value'] = 5
    if topic == 'deportes': 
        row['value'] = 6
    if topic == 'el-deportivo': 
        row['value'] = 6
    if topic == 'medio-ambiente': 
        row['value'] = 7
    if topic == 'justicia': 
        row['value'] = 8
    if topic =='reportaje-investigacion': 
        row['value'] = 8    
    if topic == 'salud': 
        row['value'] = 9 

dataset['value'].value_counts()

     10949
0      823
2      763
6      597
8      128
1       74
7       38
5       25
9       18
Name: value, dtype: int64

Tenemos muchas noticias no clasificadas, por lo que es necesario utilizar otro mecanísmo para clasificar las noticias restantes.

### 3.  Extracción de términos clave de columna title+text <a name="b"></a>

In [ ]:
matcher = Matcher(nlp.vocab)

# Pattern 1: NOUN de NOUN
pattern_1 = [{"POS": "NOUN"},{"LOWER": "de"}, {"POS": "NOUN"}]
matcher.add("NOUN-de-NOUN", [pattern_1])

# Pattern 2: NOUN ADJ
pattern_2 = [{"POS": "NOUN"}, {"POS": "ADJ"}]
matcher.add("NOUN-ADJ", [pattern_2])

# Pattern 3: 
pattern_3 = [{"POS": "NOUN"}]
matcher.add("NOUN", [pattern_3])



    Bloque de alto coste computacional ⬇️

In [ ]:
dataset['terms'] = ""
for index, row in dataset.iterrows():
    doc = nlp(row['content'].lower())

    matches = matcher(doc)
    categories = ""
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        categories = categories + span.text + "; "
    
    row['terms'] = categories
    
display(dataset.head(3))

In [ ]:
from collections import Counter
dataset["most_common"] = ""
for index, row in dataset.iterrows():
    split_it = row['terms'].split("; ")
    counter = Counter(split_it)
    row['most_common'] = counter.most_common()[0][0]

display(dataset.head(5),dataset['most_common'].value_counts())

### 4. Clasificación del dataset de entrenamiento <a name="c"></a>
Aplicaremos LDA a cada noticia y extraeremos sus tópicos, luego evaluaremos esos tópicos con las categorias deseadas.


In [ ]:
dataset['category'] = ""
#for index, row in dataset.iterrows():
#    for index in topics:

dataset

### 5.  Balance de clases dataset de entrenamiento <a name="d"></a>
Visualizamos la densidad de los tópicos con tal de combatir el sobreajuste si corresponde

In [ ]:
fig, ax = plt.subplots(1, figsize=(9,5), tight_layout=True)
ax.hist(dataset['category'],edgecolor = 'black',bins=dataset['category'].unique())
plt.xticks(rotation='vertical', fontsize=7)
plt.ylabel("densidad")
plt.show()